# Prefect Deployments - Testing & Configuration

This notebook helps you test and configure Prefect deployments before pushing to production.

**Prerequisites:**
- Prefect server running (Cloud or self-hosted)
- Environment variables configured
- Database accessible

In [ ]:
from fund_lens_etl.deployments.schedules import (
    create_bronze_deployments,
    create_silver_deployments,
    create_gold_deployments,
    create_all_deployments,
    print_schedule_summary
)

## 1. View Deployment Schedule Summary

In [ ]:
print_schedule_summary()

## 2. Create Deployment Objects

Create deployment configurations (doesn't register with Prefect yet)

In [ ]:
# Create bronze deployments
bronze_deployments = create_bronze_deployments()
print(f"Created {len(bronze_deployments)} bronze deployments:")
for dep in bronze_deployments:
    print(f"  - {dep.name}")

In [ ]:
# Create silver deployments
silver_deployments = create_silver_deployments()
print(f"Created {len(silver_deployments)} silver deployments:")
for dep in silver_deployments:
    print(f"  - {dep.name}")

In [ ]:
# Create gold deployments
gold_deployments = create_gold_deployments()
print(f"Created {len(gold_deployments)} gold deployments:")
for dep in gold_deployments:
    print(f"  - {dep.name}")

## 3. Inspect Deployment Configuration

View details of a specific deployment before deploying

In [ ]:
# Inspect bronze daily incremental deployment
daily_incremental = bronze_deployments[0]

print(f"Name: {daily_incremental.name}")
print(f"Version: {daily_incremental.version}")
print(f"Description: {daily_incremental.description}")
print(f"Tags: {daily_incremental.tags}")
print(f"Schedule: {daily_incremental.schedule}")
print(f"Parameters: {daily_incremental.parameters}")

## 4. Deploy to Prefect

**⚠️ This will register deployments with your Prefect server!**

Make sure you're connected to the correct Prefect server/workspace.

In [ ]:
# Check Prefect connection
from prefect import get_client

async def check_prefect_connection():
    async with get_client() as client:
        health = await client.api_healthcheck()
        print(f"✓ Connected to Prefect server")
        return True

# Run check
import asyncio
await check_prefect_connection()

In [ ]:
# Deploy bronze ingestion flows
print("Deploying bronze ingestion flows...")
for deployment in bronze_deployments:
    try:
        deployment_id = deployment.apply()
        print(f"  ✓ Deployed: {deployment.name} (ID: {deployment_id})")
    except Exception as e:
        print(f"  ✗ Failed: {deployment.name} - {e}")

In [ ]:
# Deploy silver transformation flows
print("Deploying silver transformation flows...")
for deployment in silver_deployments:
    try:
        deployment_id = deployment.apply()
        print(f"  ✓ Deployed: {deployment.name} (ID: {deployment_id})")
    except Exception as e:
        print(f"  ✗ Failed: {deployment.name} - {e}")

In [ ]:
# Deploy gold transformation flows
print("Deploying gold transformation flows...")
for deployment in gold_deployments:
    try:
        deployment_id = deployment.apply()
        print(f"  ✓ Deployed: {deployment.name} (ID: {deployment_id})")
    except Exception as e:
        print(f"  ✗ Failed: {deployment.name} - {e}")

## 5. Verify Deployments

List all deployments registered with Prefect

In [ ]:
from prefect.deployments import Deployment

# Note: This would need to use the Prefect API
# For now, use CLI: prefect deployment ls
print("To list deployments, run: prefect deployment ls")

## 6. Test a Deployment Run

Trigger a manual run to test (won't wait for schedule)

In [ ]:
# Example: Trigger bronze ingestion manually
# Use CLI: prefect deployment run 'bronze_ingestion/bronze-ingestion-daily-incremental'

print("To trigger a manual run:")
print("  prefect deployment run 'bronze_ingestion/bronze-ingestion-daily-incremental'")
print("\nTo monitor the run:")
print("  prefect flow-run ls")

## 7. Clean Up / Delete Deployments

If you need to remove deployments

In [ ]:
# To delete a deployment, use CLI:
print("To delete a deployment:")
print("  prefect deployment delete '<flow-name>/<deployment-name>'")
print("\nExample:")
print("  prefect deployment delete 'bronze_ingestion/bronze-ingestion-daily-incremental'")